In [ ]:
# 先查看python版本，新版本pyecharts 不兼容 python3.6以下
! python -V


Python 3.7.12


In [ ]:
# 安装pyecharts 库
! pip install pyecharts

     |████████████████████████████████| 135 kB 22.5 MB/s 
     |████████████████████████████████| 129 kB 58.7 MB/s 


In [ ]:
# 需要把excel 上传到 谷歌硬盘，命名为 book_name.xlsx

In [ ]:
# 导入需要的包
import pandas as pd
import numpy as np
from collections import Counter
import random
import pyecharts.options as opts
from pyecharts.charts import Scatter3D

In [ ]:
# 数据处理
# RGB格式颜色转换为16进制颜色格式
def RGB_to_Hex(rgb):
  color = '#'
  for i in rgb:
      num = int(i)
      # 将R、G、B分别转化为16进制拼接转换并大写  hex() 函数用于将10进制整数转换成16进制，以字符串形式表示
      color += str(hex(num))[-2:].replace('x', '0').upper()
  return color
# 生成随机颜色
def random_color():
  levels = range(32,256,32)
  return tuple(random.choice(levels) for _ in range(3))


# 从excel 读取数据
data_excel = pd.read_excel('book_name.xlsx',sheet_name='数据3D图',header = 0)
postion = data_excel["位置"].tolist()
# 已得分为x ,篮板为 y,助攻为z
x =np.array(data_excel["得分"].tolist())
y=np.array(data_excel["篮板"].tolist())
z=np.array(data_excel["助攻"].tolist())
# 统计位置出现次数
c = dict(Counter(postion))
# 获取所有位置
postion_key=list(c.keys())
# 为每个位置生成一个颜色
for key in postion_key:
  color = RGB_to_Hex(random_color())
  c[key]=color
# 记录没个位置在列表中的索引 key 位置 ，value(索引数组)
range_color=list(c.values())

# 拼接三维渲染数据
data= []
for i in range(len(x)):
  d=[]
  d.append(x[i])
  d.append(y[i])
  d.append(z[i])
  d.append(postion[i])
  d.append(i)
  data.append(d)

# 设置颜色映射，到时候一个位置设置一种颜色
pices=[]
for i in range(len(postion_key)):
  pice={}
  pice["value"]=postion_key[i]
  pice["label"]=postion_key[i]
  pice["color"]=range_color[i]
  pices.append(pice)



In [ ]:
#绘制3d散点图
(
  Scatter3D(
      init_opts=opts.InitOpts()
  )  # bg_color="black"
  .add(
      series_name="",
      # 绘制3d的数据
      data=data,
      # 指定x轴数据
      xaxis3d_opts=opts.Axis3DOpts(
          name="Score",
          type_="value",
          # textstyle_opts=opts.TextStyleOpts(color="#fff"),
      ),
      #指定y轴数据
      yaxis3d_opts=opts.Axis3DOpts(
          name="Rebound",
          type_="value",
          # textstyle_opts=opts.TextStyleOpts(color="#fff"),
      ),
      #指定z轴数据
      zaxis3d_opts=opts.Axis3DOpts(
          name="Assists",
          type_="value",
          # textstyle_opts=opts.TextStyleOpts(color="#fff"),
      ),
      grid3d_opts=opts.Grid3DOpts(width=100, height=100, depth=100),
  )
  .set_global_opts(
      #设置颜色映射
      visualmap_opts=[
          opts.VisualMapOpts(
              type_="color",
              is_calculable=True,
              is_piecewise = True,
              # 要映射颜色的维度 我们的数据是[x,y,y,地址，索引]，我们想要根据地址不同设置不同的颜色
              # 所以需要 选择第3维 ，既0，1.2，3，4
              dimension=3,
              # 图例放置位置
              pos_top="10",
              # 颜色映射规则，根据地址来设置颜色
              pieces=pices,
              # 图例水平放置
              orient = 'horizontal',
              # 颜色范围
              range_color=range_color,
              
          ),
          # 这里本来是根据某一维度得列值得大小来映射点得大小，但是我给他设置成一个固定得大小范围
          # 起到设置点大小得作用，因为我找不到其他地方设置点的大小
          opts.VisualMapOpts(
              
              type_="size",
              is_calculable=True,
              orient = 'horizontal',
              dimension=1,
              pos_bottom="10",
              max_=2.4 / 2,
              # 设置固定大小
              range_size=[20, 20],
          ),
        ]
    )
    .render()
)
# 显示3D散点图
import IPython
IPython.display.HTML(filename='/content/render.html')
